In [ ]:
#default_exp utils

# Utilities

## Delegation
> Read more about it [in this brilliant post by Jeremy Howard](https://www.fast.ai/2019/08/06/delegation/).  
> We will use it to provide the functionality of pd.DataFrame to our BaseFrame Model

In [ ]:
from nbdev.showdoc import *
from pydantic_pandas.default_standard_lib import *

A delegate is a person sent or authorized to represent others. To delegate means to entrust a task or responsibility to another person.  

The `delegates` decorator takes the named arguments from one function and injects them into the signature of the function that is being decorated. This allows for auto-completion in a jupyter notebook and add doc string templates.

In [ ]:
#export
import inspect

def delegates(to=None, keep=False):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    
    def _f(f):
        if to is None: 
            to_f,from_f = f.__base__.__init__,f.__init__
        else:          
            to_f,from_f = to,f
        
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd}
        sigd.update(s2)
        if keep: 
            sigd['kwargs'] = k
        from_f.__doc__ = f"""
        doc string from my delegate: `{to_f.__name__}`
        --------
        {to_f.__doc__}
        """
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

In [ ]:
show_doc(delegates)

<h4 id="delegates" class="doc_header"><code>delegates</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>delegates</code>(**`to`**=*`None`*, **`keep`**=*`False`*)

Decorator: replace `**kwargs` in signature with params from `to`

In [ ]:
def function_with_many_named_args(second_arg:str=None,_arg:int=None,third_arg:dt.date=None):
    """a description doc string"""
    return first_arg,second_arg,third_arg

In [ ]:
@delegates(function_with_many_named_args)
def just_one_more(first,**kwargs):

    return list(kwargs.values()) + [fourth_arg]

In [ ]:
show_doc(just_one_more)

<h4 id="just_one_more" class="doc_header"><code>just_one_more</code><a href="__main__.py#L1" class="source_link" style="float:right">[source]</a></h4>

> <code>just_one_more</code>(**`first`**, **`second_arg`**:`str`=*`None`*, **`_arg`**:`int`=*`None`*, **`third_arg`**:`date`=*`None`*)

doc string from my delegate: `function_with_many_named_args`
--------
a description doc string

In [ ]:
!nbdev_build_lib

Converted 00_core.ipynb.
Converted 98_utils.ipynb.
Converted 99_default_standard_lib.ipynb.
Converted core.ipynb.
Converted index.ipynb.
